In [1]:
import sys
sys.path.append("../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import torch
print(torch.cuda.is_available())


True


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)
training_data['x'] = torch.tensor(training_data['x']).float().to('cuda')
training_data['dx'] = torch.tensor(training_data['dx']).float().to('cuda')
training_data['t'] = torch.tensor(training_data['t']).float().to('cuda')
training_data['z'] = torch.tensor(training_data['z']).float().to('cuda')
training_data['dz'] = torch.tensor(training_data['dz']).float().to('cuda')
training_data['ddx'] = None

validation_data['x'] = torch.tensor(validation_data['x']).float().to('cuda')
validation_data['dx'] = torch.tensor(validation_data['dx']).float().to('cuda')
validation_data['t'] = torch.tensor(validation_data['t']).float().to('cuda')
validation_data['z'] = torch.tensor(validation_data['z']).float().to('cuda')
validation_data['dz'] = torch.tensor(validation_data['dz']).float().to('cuda')
validation_data['ddx'] = None


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 3
params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]
print(params['widths'][::-1])
# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 1

# training time cutoffs
params['max_epochs'] = 5
params['refinement_epochs'] = 1

[32, 64]


In [4]:
import numpy as np
import torch

def to_numpy(obj):
    if isinstance(obj, torch.Tensor):
        return obj.cpu().numpy()  # Convert tensor to numpy array
    elif isinstance(obj, dict):
        return {k: to_numpy(v) for k, v in obj.items()}  # Apply recursively to dictionaries
    elif isinstance(obj, list):
        return [to_numpy(v) for v in obj]  # Apply recursively to lists
    else:
        return obj  # Return the object as is if it's neither a tensor, list, nor dictionary


# Run training experiments

In [5]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")
    
    results_dict = train_network(training_data, validation_data, params)
    results_dict = to_numpy(results_dict)
    params = to_numpy(params)
    print(results_dict['validation_losses'][1]['decoder'])
    print(params)
    df = df._append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
Device: cuda:0


c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:180: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32).to(params['device']),
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32).to(params['device']),
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:187: UserWarn

TRAINING
Epoch 0
Epoch 0
   Training Total Loss: 0.03906350955367088
   Training decoder Loss: 0.0372636578977108
   Training sindy_z Loss: 1208.0028076171875
   Training sindy_x Loss: 17.92543601989746
   Training sindy_regularization Loss: 0.7309862375259399
   Validation Total Loss: 0.03904091939330101
   Validation decoder Loss: 0.03726543113589287
   Validation sindy_z Loss: 1243.0517578125
   Validation sindy_x Loss: 17.68178367614746
   Validation sindy_regularization Loss: 0.7309862375259399
Decoder Loss Ratio: 0.197669, Decoder SINDy Loss Ratio: 1.383683
Epoch 1
Epoch 1
   Training Total Loss: 0.0328662283718586
   Training decoder Loss: 0.031461019068956375
   Training sindy_z Loss: 231.83126831054688
   Training sindy_x Loss: 13.987504959106445
   Training sindy_regularization Loss: 0.646133542060852
   Validation Total Loss: 0.032850027084350586
   Validation decoder Loss: 0.03147069737315178
   Validation sindy_z Loss: 241.75306701660156
   Validation sindy_x Loss: 13.7286

c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_val, _, _ = autoencoder_network.define_loss(torch.tensor(train_dict['x'], dtype=torch.float32), torch.tensor(train_dict['dx'], dtype=torch.float32), params=params)


Epoch 4
   Training Total Loss: 0.0064505282789468765
   Training decoder Loss: 0.0057275001890957355
   Training sindy_z Loss: 46.7758903503418
   Training sindy_x Loss: 7.177398204803467
   Training sindy_regularization Loss: 0.5287944078445435
   Validation Total Loss: 0.009726793505251408
   Validation decoder Loss: 0.008626635186374187
   Validation sindy_z Loss: 60.284358978271484
   Validation sindy_x Loss: 10.948705673217773
   Validation sindy_regularization Loss: 0.5287944078445435
Decoder Loss Ratio: 0.045759, Decoder SINDy Loss Ratio: 0.856788


c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_val, final_losses, _ = autoencoder_network.define_loss(torch.tensor(validation_dict['x'], dtype=torch.float32), torch.tensor(validation_dict['dx'], dtype=torch.float32), params=params)
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, _, _, _, _, _, _, sindy_predict = autoencoder_network(torch.tensor(validation_dict['x'], dtype=torch.float32), torch.tens

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.